<img src="https://drive.google.com/uc?export=view&id=1Krpv420Trw6HQbArLSzrbYcpBz-9wgxw" width=300/>

# Data Engineering
## Assignment 1: Webscraping wikipedia's Billboard pages

**Alok K Pandey**<br>

## Background

    The Billboard Hot 100 is a chart that ranks the best-performing singles of the United States. Its data, published by Billboard magazine and compiled by Nielsen SoundScan, is based collectively on each single's weekly physical and digital sales, as well as airplay and streaming. At the end of a year, Billboard will publish an annual list of the 100 most successful songs throughout that year on the Hot 100 chart based on the information.
    
>> Check out Hot 100 single of 2022: https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2022

## Instructions <a class="anchor" id="instructions"></a>

<hr style="height:2pt">


# Task 1: Constructing a year-song dataframe

**Question 1:** Scrape and Parse Wikipedia for Billboard's Top 100 songs starting from 1992 to 2022.

**1.1** Scrape Wikipedia's Billboard pages from 1992 to 2022.

<details>

- Use python's `requests` module to obtain (GET) the web pages at http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1992, http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1993 till http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2022.

- Store the text from your `requests` in a dictionary called `yearstext`. 
This dictionary should have as its keys the years (as integers from 1992 to 2022), and as values corresponding to these keys the text of the page being fetched.

*Hint:* Put your requests.get() in a `for` loop and use the `time.sleep` function to wait one second between requests, you do not want Wikipedia to think you are a marauding bot attempting to mount a denial-of-service attack.

</details>


**1.2** Parse the HTML retrieved to extract ranking, song and artist information. 

  **Note:** Here are some other issues that you will need to take care of while parsing:

<details>
    
    i. The example shown above has several artists for a single song. In this case, the `band_singer` and `url` would be a list of items.

    ii. Some singles might even have multiple songs because of the way the industry works:
    ```
    {'ranking': 98,
    'song': ["You're Makin' Me High", 'Let It Flow (song)'],
    'songurl': ['/wiki/You%27re_Makin%27_Me_High', '/wiki/Let_It_Flow_(song)'],
    'titletext': '"You\'re Makin\' Me High" / "Let It Flow"',
    'band_singer': ['Toni Braxton'],
    'url': ['/wiki/Toni_Braxton']}
    ```
    (See 1997 for an example)

    iii. Some songs don't have a URL. In this case, assume there is one song in the single, set `songurl` to [`None`] and the song name to the contents of the table cell with the quotes stripped:
    ```
    {'ranking': 45,
      'song': ['Say It'],
      'songurl': [None],
      'titletext': '"Say It"',
      'band_singer': ['Voices of Theory'],
      'url': ['/wiki/Voices_of_Theory']}
    ```
    (See 1998 for an example)

</details>
<br>
<hr style="height:2pt">

In [1]:
# libraries to get you started
import numpy as np
import pandas as pd
import json
import re
import requests
from bs4 import BeautifulSoup

* Hint: Save the obtained dictionary as a json file so you do not need to run it over and over again.

In [4]:
# Start your code here
#Code for 1.1
import time

# This dict will store the retrieved web page content from the URLs for each year. Will be converted to Json later
scraped_data_yearwise = {}
for year in range(1992, 2023):
    URL = f"http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}"
    
    response = requests.get(URL)
    # status_code = 200 is for when request is successful. helps to handle errors in the future.
    if response.status_code == 200:
        scraped_data_yearwise[year] = response.text
        print(f"Data fetched successfully for {year}")
    else:
        # Handling other status codes
        print(f"Failed to fetch url data. Status code: {response.status_code}")
    # Sleep for one second between requests to avoid rate limits
    time.sleep(1)

#Code for 1.2
# this list will contain extracted specific elements from the HTML content.
parsed_data = []

# Loop through each year in 'yearstext'
for year, page_text in scraped_data_yearwise.items():
    #BeautifulSoup is being used to parse current year's HTML content to object
    content_current_year = BeautifulSoup(page_text, 'html.parser')
    
    # Find the table containing the song information
    table = content_current_year.find('table', {'class': 'wikitable'})

    # Extract rows from the table
    rows = table.find_all('tr')

    # Loop through each row (skipping the header row)
    for row in rows[1:]:
        # Get the ranking, song title, and artist(s)
        cells = row.find_all('td')
        
        if len(cells) >= 3:
            ranking = int(cells[0].text.strip())
            title = cells[1].text.strip()
            artist_text = cells[2].text.strip()

            # Handle multiple songs and artists
            song = title.strip('"').split(" / ")
            artist = artist_text.split(", ")

            # Extract URLs for songs and artists
            song_urls = [cell.find('a')['href'] if cell.find('a') else None for cell in row.find_all('td')[1:]]
            artist_urls = [cell.find('a')['href'] if cell.find('a') else None for cell in row.find_all('td')[2:]]

            # Create a dictionary with the extracted information
            song_info = {
                'year': year,
                'ranking': ranking,
                'song': song,
                'songurl': song_urls,
                'band_singer': artist,
                'url': artist_urls
            }
            
            parsed_data.append(song_info)

#saving to json from dict
json_data = json.dumps(parsed_data, indent=4)  # Convert with indentation for readability

# Save to a JSON file
with open("parsed_data.json", "w") as json_file:
    json.dump(parsed_data, json_file, indent=4)  # Write to file with indentation
print("Data saved as JSON successfully. Name: parsed_data.json")

Data fetched successfully for 1992
Data fetched successfully for 1993
Data fetched successfully for 1994
Data fetched successfully for 1995
Data fetched successfully for 1996
Data fetched successfully for 1997
Data fetched successfully for 1998
Data fetched successfully for 1999
Data fetched successfully for 2000
Data fetched successfully for 2001
Data fetched successfully for 2002
Data fetched successfully for 2003
Data fetched successfully for 2004
Data fetched successfully for 2005
Data fetched successfully for 2006
Data fetched successfully for 2007
Data fetched successfully for 2008
Data fetched successfully for 2009
Data fetched successfully for 2010
Data fetched successfully for 2011
Data fetched successfully for 2012
Data fetched successfully for 2013
Data fetched successfully for 2014
Data fetched successfully for 2015
Data fetched successfully for 2016
Data fetched successfully for 2017
Data fetched successfully for 2018
Data fetched successfully for 2019
Data fetched success

<hr style="height:2pt">

**Question 2:** Construct a DataFrame from parsed data

**2.1** Construct a dataframe from the dictionary `yearinfo`.

<details>
    
- Construct a dataframe from the dictionary created in the previous section `yearinfo`. Name this dataframe `billboardtop`.<br><br>
  Keep in mind, in the data structure we have so far, a given key can have a list of values with multiple entries. Also, our data is grouped by year. So we need a way to flatten this data into a format that will create a useful DataFrame. 
  Your final dataframe `billboardtop` should look something like this:

  <img src="https://drive.google.com/uc?export=view&id=1adDiuSmHXR7B7YO2ZNT_OJ9QejUyuJAC" width=1500/>

- Ensure that all lists in your dictionary are in different rows. <br>
  For example, a single containing two artists should be two different rows:
  <img src="https://drive.google.com/uc?export=view&id=1hxGXmP20vJ5i1N7WDg_pZBYly7gX2Gz0" width=1500/>

  A single containing two titles (1997, Rank 98) should be two different rows:
  <img src="https://drive.google.com/uc?export=view&id=1nRrDQOixcH4HfahfA9YF7R7brm_OyD7d" width=1500/>

</details>

**2.2** Check your dataframes data types and convert them to the correct data types if needed.

- Check dataframe data types using ```dtypes```.
- Convert them to the correct data types if needed using the ```astype()``` function.

**2.3** Store this dataframe in ADLS so that you can use it for tasks ahead.

* Hint: Use pickle to save you data, it retains your given datatypes and few other metadata

In [5]:
# Start your code here
import pickle

# Loading JSON data: parsed_data.json
with open("parsed_data.json", "r") as json_file:
    parsed_data = json.load(json_file)

# Create a DataFrame from the parsed data
billboardtop = pd.DataFrame(parsed_data)

# Ensure that lists within the dictionary are in different rows (flattening)
billboardtop = billboardtop.explode("song")  # Splitting the 'song' list into separate rows
billboardtop = billboardtop.explode("songurl")  # Splitting 'songurl' list into separate rows
billboardtop = billboardtop.explode("band_singer")  # Splitting 'band_singer' into separate rows
billboardtop = billboardtop.explode("url")  # Splitting 'url' into separate rows

# Convert data types if needed
billboardtop["ranking"] = billboardtop["ranking"].astype(int)  # Ensure 'ranking' is int
billboardtop["year"] = billboardtop["year"].astype(int)  # Ensure 'year' is int

# Reset the index to ensure proper DataFrame structure
billboardtop.reset_index(drop=True, inplace=True)

# Save the DataFrame as a pickle file for storage
with open("billboardtop.pickle", "wb") as pickle_file:
    pickle.dump(billboardtop, pickle_file)

print("DataFrame 'billboardtop' constructed and saved as 'billboardtop.pickle'.")
print(billboardtop.head())

DataFrame 'billboardtop' constructed and saved as 'billboardtop.pickle'.
   year  ranking             song                                   songurl  \
0  1992        1  End of the Road  /wiki/End_of_the_Road_(Boyz_II_Men_song)   
1  1992        1  End of the Road                         /wiki/Boyz_II_Men   
2  1992        2    Baby Got Back                       /wiki/Baby_Got_Back   
3  1992        2    Baby Got Back                       /wiki/Sir_Mix-a-Lot   
4  1992        3             Jump              /wiki/Jump_(Kris_Kross_song)   

     band_singer                  url  
0    Boyz II Men    /wiki/Boyz_II_Men  
1    Boyz II Men    /wiki/Boyz_II_Men  
2  Sir Mix-a-Lot  /wiki/Sir_Mix-a-Lot  
3  Sir Mix-a-Lot  /wiki/Sir_Mix-a-Lot  
4     Kris Kross     /wiki/Kris_Kross  


# Part B: Constructing a year-song-singer dataframe

Now, we need to fetch information about the singers or bands for all the songs we have in the `billboardtop` dataframe.

**Question 1:** Scrape and Parse Wikipedia for information about Artists and Bands

**1.1** Scrape the artist's Wikipedia webpages.

<details>

Since we have hundreds of artists webpages to scrape, we have created a function which implements caching in order to speed up this process.

The cache object urlcache that will avoid redundant HTTP requests (e.g. an artist might have multiple singles on a single year, or be on the list over a span of years). **Remember that this function is designed to be used in a loop over years, and then a loop over songs per year.** Since network requests are relatively slow, if we have already requested for a singer or band's wikipedia page, caching the results is a smart thing to do.

Notice that we have wrapped the call in an exception block. If the request gets an HTTP code different from 200, the cells for that URL will have a value of 1; and if the request completely fails (e.g. no network connection) the cell will have a value of 2. This will allow you to analyse the failed requests.

</details>

**1.2** Parse the HTML retrieved to extract genre of the artist, date of birth, years active and other artist information.

<details>

- Write a function `singer_band_info(url, page_text)` that returns a dictionary. 

  Here `url` should be the the url corresponding to the singer's Wikipedia page (same as the previous dataframe `billboardtop`), and page_text should be the HTML text for the corresponding artist's webpage. This function should return a dictionary which contains the following information:

  1. The genres of the band or singer. These genres should be urls, to ensure their uniqueness. Create a list, `genres`, of these urls. If there are no genres, use `['NA']`.

  2. If the page has the text "Born", extract the element with the class `.bday`. If there is no "Born", store `False`. Store either of these into the variable `born`. 

  3. If the text "Years active" is found, but there is no birthday, assume a band. Store the years active into the variable `ya`, or `False` if the text is not found. 

  The dictionary returned should be of the form:
  ```
  { 'url': '/wiki/Boyz_II_Men', 
  'genres': ['/wiki/Contemporary_R%26B_music', '/wiki/Soul_music', '/wiki/New_jack_swing'], 
  'born': None, 
  'ya': '1987–present'}
  ```
- Once the above function is created, generate a list `singer_band_info_list` to store the information extracted above. `singer_band_info_list` should be a list of the dictionaries that the function `singer_band_info` returns. The list should look something like this:
```
  'genres': ['/wiki/Contemporary_R%26B_music',
   '/wiki/Soul_music',
   '/wiki/New_jack_swing'],
  'url': '/wiki/Boyz_II_Men',
  'ya': '1987–present'},
 {'born': None,
  'genres': ['/wiki/Pop_music',
   '/wiki/Electronica_music',
   '/wiki/Dance_music',
   '/wiki/Rave_music',
   '/wiki/House_music'],
  'url': '/wiki/KWS_(band)',
  'ya': '1991–1994'},
 ... and so on]
  ```
<br>

  **Note:** Wikipedia has changed it's format along the years! So observing one artist's webpage and building your function based on it will probably give you tons of errors. Here are a few issues to remember while parsing:

    1. There are several artists that take a sabbatical between their active years (https://en.wikipedia.org/wiki/Tony!_Toni!_Ton%C3%A9!). To get the right data, write a function to calculate the longest period of time they were active and consider that as your variable `years active`. In the example give, this would be 2003–present.
    2. Birthday's are given in different formats for different pages. For example - https://en.wikipedia.org/wiki/Sir_Mix-a-Lot and https://en.wikipedia.org/wiki/Ed_Sheeran have different formats. To ensure that you get the right day, look for the 'span' tag with a 'bday' tag and ensure that there are no paranthesis around the extracted text.
    3. Year's active are also given in different formats. For example - https://en.wikipedia.org/wiki/Boyz_II_Men and https://en.wikipedia.org/wiki/Ed_Sheeran are different. You could use regex ("[0-9]{4}[–][0-9]{4}" and "[0-9]{4}[–][0-9]{4}") to ensure you are getting the right years.

  Definitely do look at your outputs as you are parsing as it can identify several edge cases you have not considered in your code.

</details>

**Question 2:** Construct a DataFrame from parsed data

**2.1** Construct a dataframe from the list `singer_band_info_list` and convert them to the correct data types if needed.

<details>

- Construct a dataframe from the list created in the previous section yearinfo. Name this dataframe `singerbandinfo`.

  Your dataframe `singerbandinfo` should look something like this:

  <img src="https://drive.google.com/uc?export=view&id=1LZh_J-LsB2p9UW7lCJaSbPNMTH-Bv61p" width=1500/> 

- Check dataframe data types using dtypes.
- Convert them to the correct data types if needed using the astype() function.


</details>

**2.2** Merge `billboardtop` and `singerbandinfo` to create one dataframe.

<details>

- Merge the artist/song data frames into one large dataframe named `finaldf` on url. Your  dataframe should look something like this:
  <img src="https://drive.google.com/uc?export=view&id=1JMoV2gvIpIQ4tDGwidT7eAGKGusF8V0Z" width=1500/> 

  Note that this has an effect of imputing to a song all the genres that the artist is active in. We know that this is not true, but it is the simplest assumption we can make, and is probably good for most artists.

</details>

### Solutions

#### Question 1: Scrape and Parse Wikipedia for information about Artists and Bands

**1.1** Scrape the artist's Wikipedia webpages.

* Hint: Before you apply uour function to the dataframe, sort `billboardtop` by year. This will ensure that we will hit the cache most as singers who show up repeatedly in the rankings will have their information already pulled.

This is optional from the perspective to optimization, you can choose to ignore it but then you will have higher run time at your end and you will end up writing more code to maintain data sanity.

In [6]:
urlcache={}
def get_page(url):
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        try:
            r = requests.get("http://en.wikipedia.org%s" % url)
            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

In [7]:
# Here I am populating the url cache
# Note that this function will take around 20 minutes to run as we are requesting for several pages
# This function is designed to be run again and again: it just tries to make sure that there are no unresolved pages left. 
billboardtop["url"].apply(get_page)

In [ ]:
# Make sure that there are no unresolved pages
# The sum below should be 0, and the boolean True. If that is not the case, run the above cell again until you get a sum of 0 and a boolean True
print (np.sum([(urlcache[k]==1) or (urlcache[k]==2) and isinstance(k,str) for k in urlcache]))
print ("Did we get all the URLs?",len(billboardtop.url.unique())==len(urlcache))

In [ ]:
# Saving the `urlcache` and removin the old object. 
keys_values = urlcache.items()
urlcache = {str(key): str(value) for key, value in keys_values}
with open("artistinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

In [ ]:
# Loading artist info
with open("artistinfo.json") as json_file:
    urlcache = json.load(json_file)

In [ ]:
len(urlcache)

**1.2** Parse the HTML retrieved to extract genre of the artist, date of birth, years active and other artist information.

Before parsing, it is important to note that Wikipedia has defined the same genre in a few different ways. Your parsing code will pick these up as different and new as they all differ with the alphabet case or an underscore instead of a hyphen.

I am adding potential duplicates list to make the task a little easier so that the above mentioned issue does not create duplicate data under same categories

In [ ]:
genres_duplicates= {'/wiki/Adult_Contemporary_music':'/wiki/Adult_contemporary',
 '/wiki/Adult_contemporary_music':'/wiki/Adult_contemporary',
'/wiki/Afrobeat':'/wiki/Afrobeats',
'/wiki/Alternative_rock':'/wiki/Alternative_Rock',
'/wiki/Avant-garde':'/wiki/Avant-garde_music',
'/wiki/Blues':'/wiki/Blues_music',
'/wiki/Comedy_hip-hop':'/wiki/Comedy_hip_hop',
'/wiki/Contemporary_R%26B':'/wiki/Contemporary_R%26B_music',
'/wiki/Contemporary_folk':'/wiki/Contemporary_folk_music',
'/wiki/Country_Folk':'/wiki/Country_folk',
'/wiki/Dance_pop':'/wiki/Dance-pop',
'/wiki/East_Coast_hip_hop':'/wiki/East_coast_hip_hop',
'/wiki/Electronic_Dance_Music':'/wiki/Electronic_dance_music',
'/wiki/Electronica':'/wiki/Electronica_music',
'/wiki/Emo':'/wiki/Emo_music',
'/wiki/Electropop':'/wiki/Electro-pop',
'/wiki/Folk-pop':'/wiki/Folk_pop',
'/wiki/Funk':'/wiki/Funk_music',
'/wiki/Grime_(music_genre)':'/wiki/Grime_music',
'/wiki/Gangsta_Rap':'/wiki/Gangsta_rap',
'/wiki/Hip_Hop_music': '/wiki/Hip_hop','/wiki/Hip_hop_music':'/wiki/Hip_hop',
'/wiki/Hyphy':'/wiki/Hyphy_music',
'/wiki/Latin_music':'/wiki/Latin_music_(genre)',
'/wiki/West_Coast_hip_hop':'/wiki/West_coast_hip_hop',
'/wiki/Southern_Hip_Hop':'/wiki/Southern_hip_hop',
'/wiki/Ska':'/wiki/Ska_music',
'/wiki/Pop-rock':'/wiki/Pop_rock',
'/wiki/Pop_Music':'/wiki/Pop_music',
'/wiki/Nu_metal':'/wiki/Nu_metal_music',
'/wiki/Hard_Rock':'/wiki/Hard_rock',
'/wiki/Pop_Rock':'/wiki/Pop_rock',
'/wiki/Post-Grunge':'/wiki/Post-grunge',
'/wiki/SoundCloud_rap':'/wiki/Soundcloud_rap'}

def genre_duplicates(genres):
    for i in range(len(genres)):
        if genres[i] in genres_duplicates.keys():
            genres[i]=genres_duplicates[genres[i]]
    return genres 

**1.2.1** Define a function to calculate the longest active years

In [ ]:
# Start your code here
import re

# Goal: to calculate longest active years from a list of active year strings
def longest_active_years(active_years):
    # Extract all the spans of years from the given active_years string(s)
    spans = re.findall(r"\d{4}–\d{4}", active_years)

    # If there are multiple spans, find the longest one
    longest_span = None
    longest_duration = 0
    
    for span in spans:
        start_year, end_year = map(int, span.split('–'))
        duration = end_year - start_year
        
        if duration > longest_duration:
            longest_duration = duration
            longest_span = span
    
    return longest_span


**1.2.2** Please write the function `singer_band_info` according to the following specification.

In [ ]:
"""
Function
--------
singer_band_info

Inputs
------
url: the url
page_text: the text associated with the url
   
Returns
-------
A dictionary with the following data:
    url: copy the input argument url into this value
    genres: the genres that the band or singer works in
    born: the artist's birthday
    ya: years active variable

Notes
-----
See description above. Also note that some of the genres urls might require a 
bit of care and special handling.
"""
# Start your code here

def singer_band_info(url, page_text):
    # Create a BeautifulSoup object to parse the HTML text
    soup = BeautifulSoup(page_text, "html.parser")
    
    # Initialize the dictionary with the URL
    info = {
        "url": url,
        "genres": [],
        "birth": None,
        "years_active": None
    }
    
    # Extract genres
    genres = []
    genre_elements = soup.find_all("a", href=re.compile("/wiki/"))
    for element in genre_elements:
        href = element.get("href", "")
        if "/wiki/Category:" not in href:  # Avoid category pages
            genres.append(href)
    
    # Clean up genres to handle duplicates or variations
    info["genres"] = list(set(genre_duplicates(genres)))
    
    # Extract the "Born" date
    born_element = soup.find("span", class_="bday")
    if born_element:
        info["birth"] = born_element.get_text()
    else:
        info["birth"] = None  # Default if no "Born" date found
    
    # Extract the years active
    active_text = soup.find(text=re.compile("Years active"))
    if active_text:
        parent = active_text.parent
        if parent:
            # Use regex to find the span of years
            year_match = re.search(r"(\d{4}–\d{4}|\d{4}–present)", parent.text)
            if year_match:
                info["years_active"] = year_match.group(0)
            else:
                info["years_active"] = None
    else:
        info["years_active"] = None
    
    return info


#### Question 2: Construct a DataFrame from parsed data

**2.1** Construct a dataframe from the list `singer_band_info_list`.

In [ ]:
# Start your code here
# list to store info about singers & bands
singer_band_info_list = []

for url in billboardtop['url']:
    url_complete = f"https://en.wikipedia.org{url}" 
    response = requests.get(url_complete)

    # If the request is successful, process the information
    if response.status_code == 200:
        page_text = response.text
        # usage of singer_band_info function here:
        get_info = singer_band_info(url_complete, page_text)
        singer_band_info_list.append(get_info)
    else:
        # If not successful, use default values
        singer_band_info_list.append({'url': url_complete, 'genres': ['NA'], 'born': None, 'years_active': None})

print(singer_band_info_list.head())
with open('singer_band_info_list.json', 'w') as file:
    json.dump(singer_band_info_list, file)
print("Dumping singer_band_info_list for uninterrupted usage later adn to get dataframe from it")

**2.2** Merge billboardtop and singerbandinfo to create one dataframe.

In [ ]:
# Start your code here
# Construct the singerbandinfo DataFrame from the list of dictionaries
singerbandinfo_df = pd.DataFrame(singer_band_info_list)

# Merge the two dataframes on the 'url' column
final_data = pd.merge(billboardtop, singerbandinfo_df, on='url', how='inner')

# Display the merged dataframe to confirm the structure
print(final_data.head())


# Part C: Exploratory Data Analysis (EDA)

**Question 1:** What has been the trajectory of various genres in the popular zeitgeist?<br>

**1.1** What are the 30 most popular genres?<br>

<details>

1.1.1 Find the top 30 genres and plot a bar plot of these genres.<br>
1.1.2 Feel free to plot any other visualizations that you can think of!<br>
1.1.3 Calculate the mean of the dataframe and eliminate the first two columns (`year` and `ranking`) to get means of all the genre columns.<br>
1.1.4 Sort it in ascending order and pick the top 30.<br>

</details>

**1.2** How has the popularity of these 30 genres changed with time?<br>

<details>

1.2.1 Create a subframe of the ranking and year for each genre.<br>
1.2.2 Groupby() function to group by year to create a dataframe that contains the rankings of every song from that genre in a given year.<br>

</details>

**Question 2:** Who are the highest quality singers?<br>

**2.1** Who are the most occurring artists in Billboard's Top 100 list?<br>

<details>

2.1.1 Count the number of times a singer appears in the top 100 over a certain time period. Consider an artist appearing twice in a year as two appearances.<br>

2.1.2 Plot a bar chart of the artists who have occurred at least more than 15 times in the given time frame.<br>

</details>



**2.3**  What is the age at which singers achieve their top ranking?<br>

<details>

* Plot a histogram of the age at which artists reach their top ranking.<br>

</details>

**2.4** At what year since inception do bands reach their top rankings?<br>

<details>

* Make a similar calculation to plot a histogram of the years since inception at which bands reach their top ranking.<br>
    
</details>

### Solutions

In [ ]:
# Start your code here
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Given final_data (merged DataFrame from previous steps)

# -- Question 1: Trajectory of Various Genres in the Popular Zeitgeist --

# **1.1** Goal: to find the top 30 genres and plot a bar plot
# Extracted all unique genres from 'genres' column and count their occurrences
all_genres = []
for genres in finaldf['genres']:
    if genres:  # Make sure it's not empty
        all_genres.extend(genres)

# Create a Series with genre counts
genre_counts = pd.Series(all_genres).value_counts()

# Get the top 30 genres and plot a bar plot
top_30_genres = genre_counts.head(30)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_30_genres.values, y=top_30_genres.index, orient='h')
plt.title("Top 30 Most Popular Genres")
plt.xlabel("Number of Occurrences")
plt.show()


In [ ]:
# **1.2** How has the popularity of these 30 genres changed with time?

# Create a dataframe to hold the ranking and year for each genre
genre_years = pd.DataFrame()

# Append rankings and years for each genre
for genre in top_30_genres.index:
    genre_years = pd.concat([genre_years, finaldf[['year', 'ranking']].loc[finaldf['genres'].apply(lambda g: genre in g)]])

# Group by year and calculate the mean ranking for each genre
genre_trends = genre_years.groupby('year').mean().sort_index()

# Plot the trend of popularity for the top 30 genres over time
plt.figure(figsize=(12, 8))
sns.lineplot(data=genre_trends, x='year', y='ranking')
plt.title("Popularity Trend of Top 30 Genres Over Time")
plt.xlabel("Year")
plt.ylabel("Average Ranking")
plt.show()

In [ ]:
# -- Question 2: Who Are the Highest Quality Singers? --

# **2.1** Who are the most occurring artists in Billboard's Top 100 list?
# Count the number of times an artist appears in the top 100 over the given time period
artist_occurrences = finaldf['artist'].value_counts()

# Plot a bar chart of artists who have occurred at least 15 times in the given time frame
min_occurrences = 15
artists_above_threshold = artist_occurrences[artist_occurrences > min_occurrences]

plt.figure(figsize=(12, 8))
sns.barplot(x=artists_above_threshold.values, y=artists_above_threshold.index, orient='h')
plt.title("Artists with More Than 15 Occurrences in Billboard's Top 100")
plt.xlabel("Number of Occurrences")
plt.show()

# 2.2 not given

In [ ]:

# **2.3** What is the age at which singers achieve their top ranking?
# Extract the ages and plot a histogram
finaldf['birth'] = pd.to_datetime(finaldf['birth'], errors='coerce')  # Convert 'birth' to datetime
finaldf['age_at_top'] = finaldf['year'] - finaldf['birth'].dt.year  # Calculate age at top ranking

plt.figure(figsize=(12, 8))
sns.histplot(data=finaldf, x='age_at_top', kde=True)
plt.title("Histogram of Age at Which Singers Achieve Their Top Ranking")
plt.xlabel("Age")
plt.show()

In [ ]:
# **2.4** At what year since inception do bands reach their top rankings?
# Calculate years since inception and plot a histogram
finaldf['inception'] = pd.to_datetime(finaldf['years_active'].apply(lambda ya: ya.split("–")[0]), errors='coerce')
finaldf['years_since_inception'] = finaldf['year'] - finaldf['inception'].dt.year  # Years since inception

plt.figure(figsize=(12, 8))
sns.histplot(data=finaldf, x='years_since_inception', kde=True)
plt.title("Histogram of Years Since Inception When Bands Reach Their Top Rankings")
plt.xlabel("Years Since Inception")
plt.show()